# prepare

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pyproj import Geod
import scipy

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Data Clean

In [2]:
def cutomizedCoordinationFix(df):
    df = df.assign(rev=df.dropoff_latitude<df.dropoff_longitude)
    idx = (df['rev'] == 1)
    df.loc[idx,['dropoff_longitude','dropoff_latitude']] = df.loc[idx,['dropoff_latitude','dropoff_longitude']].values
    df.loc[idx,['pickup_longitude','pickup_latitude']] = df.loc[idx,['pickup_latitude','pickup_longitude']].values
    df = df.drop(columns=['rev'])
    return df

def clean_df(df):    
    #reverse incorrectly assigned longitude/latitude values
    df = cutomizedCoordinationFix(df)
    
    return df

# Customize Feature Definition

In [3]:
def distance(lon1,lat1,lon2,lat2):
    az12,az21,dist = Geod(ellps='WGS84').inv(lon1,lat1,lon2,lat2)
    return dist

def direction(lon1,lat1,lon2,lat2):
    az12,az21,dist = Geod(ellps='WGS84').inv(lon1,lat1,lon2,lat2)
    return az12

def convertToDateTime(s):
    s = s.str.slice(0, 15)
    return pd.to_datetime(s, utc=True, format='%Y-%m-%d %H:%M')

def date_features(df):
    df = df.assign(
        #time features
        year=df.pickup_datetime.dt.year,
        dayofyear=df.pickup_datetime.dt.dayofyear,
        weekday=df.pickup_datetime.dt.dayofweek,
        time=(df.pickup_datetime.dt.hour * 60+df.pickup_datetime.dt.minute)
    )
    return df

def location_features(df):
    pickup_long = df.pickup_longitude.tolist()
    pickup_lat = df.pickup_latitude.tolist()
    dropoff_long = df.dropoff_longitude.tolist()
    dropoff_lat = df.dropoff_latitude.tolist()
    
    df = df.assign(
        #distance between pickup and dropoff, and bearing from pickup to dropoff
        distance=distance(pickup_long, pickup_lat, dropoff_long, dropoff_lat),
        direction=direction(pickup_long, pickup_lat, dropoff_long, dropoff_lat)
    )
    
    return df

def modelling_features(df):    
    # format datetime to minute
    df['pickup_datetime'] = convertToDateTime(df['pickup_datetime'])
    
    df = date_features(df)
    df = location_features(df)
    
    # drop off original features
    df = df.drop(columns=['pickup_datetime'])
    
    return df

# util func define

In [4]:
def get_split_sets(train):
    x = train.drop(columns=['fare_amount'])
    y = train['fare_amount'].values
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=0)
    return x_train, x_val, y_train, y_val

# Pipeline

In [5]:
from utils import Timer
from IPython.display import display

path = "../data/"
TRAIN_PATH = f'{path}/train.csv'
TEST_PATH = f'{path}/test.csv'

cols = [
    'fare_amount', 'pickup_datetime','pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude', 'passenger_count'
]
with Timer("Load test"):
    test = pd.read_csv(TEST_PATH)
    
#sampled_line = 10000000
with Timer(f"Load train full"):
    train = pd.read_csv(TRAIN_PATH, usecols=cols)

print("Raw data")
display(train)

with Timer("Data Wrangling for train"):
    train = clean_df(train)
    
with Timer("enrich feature for train"):
    print(f"\n*** before enrich ***")
    print(train.dtypes)
    
    train = modelling_features(train)
    
    print(f"\n*** after enrich ***")
    print(train.dtypes)
     
print("Before Training")
display(train)


Load test took 0.028385845012962818 sec
Load train full took 55.62999694980681 sec
Raw data


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5000,2009-06-15 17:26:21 UTC,-73.8443,40.7213,-73.8416,40.7123,1
1,16.9000,2010-01-05 16:52:16 UTC,-74.0160,40.7113,-73.9793,40.7820,1
2,5.7000,2011-08-18 00:35:00 UTC,-73.9827,40.7613,-73.9912,40.7506,2
3,7.7000,2012-04-21 04:30:42 UTC,-73.9871,40.7331,-73.9916,40.7581,1
4,5.3000,2010-03-09 07:51:00 UTC,-73.9681,40.7680,-73.9567,40.7838,1
...,...,...,...,...,...,...,...
55423851,14.0000,2014-03-15 03:28:00 UTC,-74.0053,40.7400,-73.9633,40.7626,1
55423852,4.2000,2009-03-24 20:46:20 UTC,-73.9578,40.7655,-73.9516,40.7740,1
55423853,14.1000,2011-04-02 22:04:24 UTC,-73.9705,40.7523,-73.9605,40.7973,1
55423854,28.9000,2011-10-26 05:57:51 UTC,-73.9809,40.7646,-73.8706,40.7740,1


Data Wrangling for train took 4.903328228276223 sec

*** before enrich ***
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

*** after enrich ***
fare_amount          float64
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
year                   int64
dayofyear              int64
weekday                int64
time                   int64
distance             float64
direction            float64
dtype: object
enrich feature for train took 171.68478156579658 sec
Before Training


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,dayofyear,weekday,time,distance,direction
0,4.5000,-73.8443,40.7213,-73.8416,40.7123,1,2009,166,0,1022,1029.6007,167.1927
1,16.9000,-74.0160,40.7113,-73.9793,40.7820,1,2010,5,1,965,8443.4414,21.5739
2,5.7000,-73.9827,40.7613,-73.9912,40.7506,2,2011,230,3,3,1389.1322,-148.8690
3,7.7000,-73.9871,40.7331,-73.9916,40.7581,1,2012,112,5,243,2795.7897,-7.7015
4,5.3000,-73.9681,40.7680,-73.9567,40.7838,1,2010,68,1,425,1998.3378,28.8962
...,...,...,...,...,...,...,...,...,...,...,...,...
55423851,14.0000,-74.0053,40.7400,-73.9633,40.7626,1,2014,74,5,182,4339.8703,54.7851
55423852,4.2000,-73.9578,40.7655,-73.9516,40.7740,1,2009,83,1,1204,1070.1565,28.9918
55423853,14.1000,-73.9705,40.7523,-73.9605,40.7973,1,2011,92,5,1320,5069.4491,9.5518
55423854,28.9000,-73.9809,40.7646,-73.8706,40.7740,1,2011,299,2,305,9369.5665,83.6125


In [6]:
train.to_parquet("low_feat_process_nyc_taxi.parquet")

In [2]:
train = pd.read_parquet("low_feat_process_nyc_taxi.parquet")

# EvalML Train

In [4]:
from utils import Timer

def get_split_sets(train):
    x = train.drop(columns=['fare_amount'])
    y = train['fare_amount'].values
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=123)
    return x_train, x_val, y_train, y_val

with Timer("split train and val"):
    x_train, x_val, y_train, y_val = get_split_sets(train)
    
# looking for right ml pipeline
import evalml
from evalml import AutoMLSearch

automl = AutoMLSearch(X_train=x_train,
                      y_train=y_train,
                      X_holdout=x_val,
                      y_holdout=y_val,
                      problem_type="regression",
                      objective="root mean squared error",
                      verbose=True,)
automl.search()

best_pipeline = automl.best_pipeline
with Timer("train"):
    best_pipeline.fit(x_train, y_train)
    
best_pipeline.score(X_val, y_val, objectives=["root mean squared error"])

split train and val took 19.074030242860317 sec
2022-11-30 03:27:24,585 featuretools - WARNING    Featuretools failed to load "nlp_primitives" primitives from "nlp_primitives". For a full stack trace, set logging to debug.
2022-11-30 03:27:24,601 featuretools - WARNING    Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.


NameError: name 'X_val' is not defined

# LGBM Train

In [9]:
def get_split_sets(train):
    x = train.drop(columns=['fare_amount'])
    y = train['fare_amount'].values
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=0)
    return x_train, x_val, y_train, y_val

with Timer("split train and val"):
    x_train, x_val, y_train, y_val = get_split_sets(train)

params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': 4,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'max_bin' : 5000 ,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1,
        'zero_as_missing': True,
        'seed':0,
        'num_rounds':1000,
        'num_boost_round': 10000,
        'early_stopping_rounds': 50
    }


lgbm_train = lgbm.Dataset(x_train, y_train, silent=False, categorical_feature=['passenger_count','year','time','dayofyear','weekday'])
lgbm_val = lgbm.Dataset(x_val, y_val, silent=False, categorical_feature=['passenger_count','year','time','dayofyear','weekday'])

with Timer("train"):
    model = lgbm.train(params=params, train_set=lgbm_train, valid_sets=lgbm_val, verbose_eval=100)
    
with Timer("predict"):
    pred = model.predict(x_val, num_iteration=model.best_iteration)
    
with Timer("calculate rmse"):
    rmse = np.sqrt(mean_squared_error(y_val, pred))

print('LightGBM RMSE', rmse)

split train and val took 22.421847986988723 sec
[LightGBM] [Warning] bagging_fraction is set=1, subsample=0.8 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] bagging_fraction is set=1, subsample=0.8 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.834323 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 30533
[LightGBM] [Info] Number of data points in the train set: 49881470, number of used features: 11
[LightGBM] [Warning] bagging_fraction is set=1, subsample=0.8 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Info] Start training from score 11.345439
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 4.63246
Early stopping, best iteration is:

In [12]:
with Timer("split train and val"):
    x_train, x_val, y_train, y_val = get_split_sets(train)

lgbm_params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'reg_sqrt': True,
    'learning_rate': 0.03,
    'num_leaves': 1200,
    'max_depth': -1,
    'max_bin': 5000,
    'num_rounds': 1200,
    'early_stopping_round': 50,
    'metric': 'rmse'
}

lgbm_train = lgbm.Dataset(x_train, y_train, silent=False)
lgbm_val = lgbm.Dataset(x_val, y_val, silent=False)

with Timer("train"):
    model = lgbm.train(params=lgbm_params, train_set=lgbm_train, valid_sets=lgbm_val, verbose_eval=100)
    
with Timer("predict"):
    pred = model.predict(x_val, num_iteration=model.best_iteration)
    
with Timer("calculate rmse"):
    rmse = np.sqrt(mean_squared_error(y_val, pred))

print('LightGBM RMSE', rmse)

split train and val took 3.7570005068555474 sec
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.129873 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30536
[LightGBM] [Info] Number of data points in the train set: 9000000, number of used features: 12
[LightGBM] [Info] Start training from score 3.177760
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 4.25898
[200]	valid_0's rmse: 4.12228
[300]	valid_0's rmse: 4.08928
[400]	valid_0's rmse: 4.0765
[500]	valid_0's rmse: 4.07053
[600]	valid_0's rmse: 4.06616
[700]	valid_0's rmse: 4.06431
[800]	valid_0's rmse: 4.06246
[900]	valid_0's rmse: 4.06072
[1000]	valid_0's rmse: 4.05996
[1100]	valid_0's rmse: 4.05938
Early stopping, best iteration is:
[1107]	valid_0's rmse: 4.05933
train took 803.9622147129849 sec
predict took 4.348827651701868 sec
calculate rmse

# EDA

In [ ]:
def exploration_features(df):
    """adds features for use in the EDA section"""
    df = shared_features(df)
    df = (
        df
        .assign(
            hour=df.pickup_datetime.dt.hour,
            close_to_airport='No',
            fare_per_km=df.fare_amount*1000/df.distance,
            direction_bucket = pd.cut(df.direction, np.linspace(-180, 180, 37)),

            #small location buckets
            pickup_long_bucket=pd.cut(df.pickup_longitude, bins=2550, labels=False),
            pickup_lat_bucket=pd.cut(df.pickup_latitude, bins=2200, labels=False),
            dropoff_long_bucket=pd.cut(df.dropoff_longitude, bins=2550, labels=False),
            dropoff_lat_bucket=pd.cut(df.dropoff_latitude, bins=2200, labels=False),


            #large location buckets
            pickup_long_bucket_big=pd.cut(df.pickup_longitude, bins=255, labels=False),
            pickup_lat_bucket_big=pd.cut(df.pickup_latitude, bins=220, labels=False),
            dropoff_long_bucket_big=pd.cut(df.dropoff_longitude, bins=255, labels=False),
            dropoff_lat_bucket_big=pd.cut(df.dropoff_latitude, bins=220, labels=False)
        )
        .drop(columns='pickup_datetime')
        .query("0 < distance")
    )

    df.loc[((df['pickup_dist_jfk']<1500) | (df['dropoff_dist_jfk']<1500)), 'close_to_airport'] = 'JFK'
    df.loc[((df['pickup_dist_lga']<1500) | (df['dropoff_dist_lga']<1500)), 'close_to_airport'] = 'LaGuardia'
    df.loc[((df['pickup_dist_nla']<1500) | (df['dropoff_dist_nla']<1500)), 'close_to_airport'] = 'Newark'  
    return df

